In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt
print("슝=3")

슝=3


In [3]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

print("슝=3")

슝=3


In [4]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs
print("슝=3")

슝=3


In [11]:
mha = MultiHeadAttention(d_model=512, num_heads=8)
inputs = {'query': tf.random.normal(shape=(16, 100, 512)),
          'key': tf.random.normal(shape=(16, 100, 512)),
          'value': tf.random.normal(shape=(16, 100, 512)),
          'mask': None}
outputs = mha(inputs)
print(outputs)

tf.Tensor(
[[[ 0.31194213  0.06846652 -0.14545253 ...  0.34152558 -0.01293674
   -0.10417397]
  [ 0.06113582 -0.2637791   0.13253844 ...  0.3520121   0.08305789
   -0.26265913]
  [ 0.05521496 -0.04981989 -0.02933395 ...  0.32582527 -0.00207202
    0.01891272]
  ...
  [ 0.11160279 -0.05528751 -0.08048372 ...  0.2031199  -0.07084238
   -0.1108495 ]
  [-0.03900976 -0.18911031 -0.11804104 ...  0.20212713 -0.38104144
    0.1061062 ]
  [ 0.09263898 -0.12962747 -0.06329235 ...  0.02945981 -0.17609507
   -0.03076953]]

 [[-0.14625365 -0.19078861  0.05680215 ...  0.01729532  0.09427337
    0.15989985]
  [-0.00879875  0.09307674 -0.02225625 ...  0.19250618 -0.1287268
   -0.12116873]
  [-0.18215227  0.23925833  0.37365118 ...  0.14650804 -0.00389771
    0.16848606]
  ...
  [-0.27207857  0.00303634  0.27001342 ...  0.11705796  0.16962379
    0.2031405 ]
  [-0.30680832 -0.12186713  0.30924895 ...  0.00931952 -0.22447443
    0.06137569]
  [ 0.00612278  0.2276799  -0.10122065 ...  0.20439091 -0.08018